In [1]:
import json

In [2]:
with open('prediction_conll04_30/prediction.json.tmp', 'r') as f:
    data_tmp = json.load(f)

In [3]:
with open('prediction_conll04_30/prediction.json', 'r') as f:
    data = json.load(f)

In [4]:
len(data_tmp)

1303

In [5]:
len(data)

1303

In [6]:
with open('../data/core_scierc/scierc_30/fold_1/unlabeled_w_labels.json', 'r') as f:
    unlabeled = json.load(f)

In [7]:
len(unlabeled)

1303

In [8]:
count = 0
for row in unlabeled:
    if row['relations'] == []:
        count += 1

In [9]:
count

338

In [205]:
# def convert_doc_type_to_asp_type(atype, form):
#     if form == 'entity':
#         return atype.lower()
#     else:
#         split = atype.split('_')
#         if len(split) > 1:
#             return atype.split('_')[0].lower() + atype.split('_')[1]
#         else:
#             return atype.split('_')[0].lower()

In [10]:
def convert_doc_to_atoms(dataset, pred, verbose):
    atoms = []
    entities = pred['entities']
    relations = pred['relations']
    for ent in entities:
        etype = convert_doc_type_to_asp_type(dataset, ent['type'], 'entity')
        start = ent['start']
        end = ent['end']
        prob = 1
        if 'prob' in ent:
            prob = ent['prob']
        if verbose:
            c = f'atom(entity({etype},{start},{end}),"{prob}").'
        else:
            c = f'atom(entity({etype},{start},{end})).'
        atoms.append(c)
    for rel in relations:
        rtype = convert_doc_type_to_asp_type(dataset, rel['type'], 'relation')
        head = entities[rel['head']]
        head_start, head_end = head['start'], head['end']
        tail = entities[rel['tail']]
        tail_start, tail_end = tail['start'], tail['end']
        prob = 1
        if 'prob' in rel:
            prob = rel['prob']
        if verbose:
            c = f'atom(relation({rtype},{head_start},{head_end},{tail_start},{tail_end}),"{prob}").'
        else:
            c = f'atom(relation({rtype},{head_start},{head_end},{tail_start},{tail_end})).'
        atoms.append(c)
    return atoms

In [11]:
def convert_doc_type_to_asp_type(dataset, atype, form):
    dct = {
        'conll04': {
            'entity': {
                'Peop': 'peop',
                'Other': 'other',
                'Loc': 'loc',
                'Org': 'org'
            },
            'relation': {
                'Work_For': 'workFor',
                'OrgBased_In': 'orgbasedIn',
                'Live_In': 'liveIn',
                'Located_In': 'locatedIn',
                'Kill': 'kill'
            }
        },
        'ade': {
            'entity': {
                'Adverse-Effect': 'adverseEffect',
                'Drug': 'drug'
            },
            'relation': {
                'Adverse-Effect': 'adverseEffect'
            }
        },
        'scierc': {
            'entity': {
                'Task': 'task',
                'Method': 'method',
                'Material': 'material',
                'OtherScientificTerm': 'otherScientificTerm',
                'Metric': 'metric',
                'Generic': 'generic'
            },
            'relation': {
                'Used-for': 'usedFor',
                'Feature-of': 'featureOf',
                'Hyponym-of': 'hyponymOf',
                'Evaluate-for': 'evaluateFor',
                'Part-of': 'partOf',
                'Compare': 'compare',
                'Conjunction': 'conjunction'
            }
        }
    }
    return dct[dataset][form][atype]

In [147]:
t = {"tokens": ["In", "this", "paper", ",", "a", "novel", "method", "to", "learn", "the", "intrinsic", "object", "structure", "for", "robust", "visual", "tracking", "is", "proposed", "."], "entities": [{"type": "Generic", "start": 6, "end": 7, "prob": 0.9992194175720215}, {"type": "OtherScientificTerm", "start": 10, "end": 13, "prob": 0.9956142902374268}, {"type": "Task", "start": 14, "end": 17, "prob": 0.9977304339408875}], "relations": [{"type": "Used-for", "head": 0, "tail": 1, "prob": 0.9994097948074341}, {"type": "Used-for", "head": 0, "tail": 2, "prob": 0.9996190071105957}]}

In [148]:
convert_doc_to_atoms('scierc', t)

['atom(entity(generic,6,7),"0.9992194175720215").',
 'atom(entity(otherScientificTerm,10,13),"0.9956142902374268").',
 'atom(entity(task,14,17),"0.9977304339408875").',
 'atom(relation(usedFor,6,7,10,13),"0.9994097948074341").',
 'atom(relation(usedFor,6,7,14,17),"0.9996190071105957").']

In [12]:
def debug_case(i, dataset):
    atoms = convert_doc_to_atoms(dataset, unlabeled[i], True)
    pred_atoms = convert_doc_to_atoms(dataset, data[i], True)
    revised_atoms = convert_doc_to_atoms(dataset,data_tmp[i], True)
    
    print('grtr: ', atoms)
    print('pred: ', pred_atoms)
    print('revi: ', revised_atoms)

In [13]:
debug_case(21, 'scierc')

grtr:  ['atom(entity(generic,7,8),"1").', 'atom(entity(otherScientificTerm,11,13),"1").', 'atom(entity(material,14,16),"1").', 'atom(relation(usedFor,7,8,11,13),"1").', 'atom(relation(usedFor,14,16,7,8),"1").']
pred:  ['atom(entity(generic,4,5),"0.9986431002616882").', 'atom(entity(generic,6,8),"0.5896682739257812").', 'atom(entity(generic,7,8),"0.9957417249679565").', 'atom(entity(otherScientificTerm,11,13),"0.8259739279747009").', 'atom(entity(material,14,16),"0.9959904551506042").', 'atom(relation(conjunction,4,5,6,8),"0.9449558854103088").', 'atom(relation(usedFor,4,5,11,13),"0.5559952855110168").', 'atom(relation(usedFor,6,8,7,8),"0.9140499830245972").', 'atom(relation(usedFor,6,8,11,13),"0.9741834402084351").', 'atom(relation(usedFor,7,8,6,8),"0.9727578163146973").', 'atom(relation(usedFor,7,8,11,13),"0.9885531663894653").', 'atom(relation(usedFor,14,16,11,13),"0.9740813970565796").']
revi:  ['atom(entity(generic,4,5),"1").', 'atom(entity(generic,7,8),"1").', 'atom(entity(otherSc

In [21]:
count = 0
for i in range(len(unlabeled)):
    atoms = convert_doc_to_atoms('scierc', unlabeled[i], False)
    
    pred_atoms = convert_doc_to_atoms('scierc', data[i], True)
    
    revised_atoms = convert_doc_to_atoms('scierc', data_tmp[i], False)
    
    print('------------------------')
    print(i)
    print('grtr: ', atoms)
    print('pred: ', pred_atoms)
    print('revi: ', revised_atoms)
#     print('grtr == pred: ', set(atoms) == set(pred_atoms))
    print('revi != pred: ', set(revised_atoms) != set(pred_atoms))
    
#     if set(revised_atoms) != set(pred_atoms):
#         count += 1
        
#     if len(set(revised_atoms)) != len(set(pred_atoms)):
#         print('------------------------')
#         print(i)
#         count += 1
#         print('grtr: ', atoms)
#         print('pred: ', pred_atoms)
#         print('revi: ', revised_atoms)
#     print('grtr == revi: ', set(atoms) == set(revised_atoms))

------------------------
0
grtr:  ['atom(entity(material,0,1)).', 'atom(entity(otherScientificTerm,10,11)).', 'atom(entity(otherScientificTerm,17,21)).']
pred:  ['atom(entity(material,0,1),"0.9947904348373413").', 'atom(entity(otherScientificTerm,5,6),"0.6586177945137024").', 'atom(entity(otherScientificTerm,10,11),"0.9142281413078308").', 'atom(entity(otherScientificTerm,18,21),"0.9858071804046631").']
revi:  ['atom(entity(material,0,1)).', 'atom(entity(otherScientificTerm,5,6)).', 'atom(entity(otherScientificTerm,10,11)).', 'atom(entity(otherScientificTerm,18,21)).']
revi != pred:  True
------------------------
1
grtr:  ['atom(entity(generic,1,2)).', 'atom(entity(otherScientificTerm,7,8)).', 'atom(entity(otherScientificTerm,9,11)).', 'atom(entity(otherScientificTerm,20,22)).', 'atom(entity(material,23,24)).', 'atom(entity(otherScientificTerm,26,28)).', 'atom(entity(material,29,30)).', 'atom(entity(material,32,33)).', 'atom(relation(conjunction,7,8,9,11)).', 'atom(relation(featureOf,2

grtr:  ['atom(entity(material,1,3)).']
pred:  ['atom(entity(otherScientificTerm,1,3),"0.4772930145263672").', 'atom(entity(method,7,9),"0.9299885034561157").']
revi:  ['atom(entity(otherScientificTerm,1,3)).', 'atom(entity(method,7,9)).']
revi != pred:  True
------------------------
324
grtr:  ['atom(entity(material,0,4)).', 'atom(entity(method,11,12)).', 'atom(entity(generic,14,15)).', 'atom(relation(usedFor,0,4,11,12)).']
pred:  ['atom(entity(material,2,4),"0.8546357750892639").', 'atom(entity(method,11,12),"0.881661593914032").', 'atom(entity(generic,14,15),"0.9918076395988464").', 'atom(relation(usedFor,2,4,11,12),"0.9988197684288025").', 'atom(relation(usedFor,14,15,11,12),"0.7213429808616638").']
revi:  ['atom(entity(material,2,4)).', 'atom(entity(method,11,12)).', 'atom(entity(generic,14,15)).', 'atom(relation(usedFor,2,4,11,12)).', 'atom(relation(usedFor,14,15,11,12)).']
revi != pred:  True
------------------------
325
grtr:  ['atom(entity(material,7,8)).', 'atom(entity(generic

pred:  ['atom(entity(method,0,2),"0.6784198880195618").', 'atom(entity(task,9,12),"0.9970532655715942").', 'atom(relation(usedFor,0,2,9,12),"0.9886348247528076").']
revi:  ['atom(entity(method,0,2)).', 'atom(entity(task,9,12)).', 'atom(relation(usedFor,0,2,9,12)).']
revi != pred:  True
------------------------
663
grtr:  ['atom(entity(method,4,6)).', 'atom(entity(material,8,10)).', 'atom(entity(material,17,19)).', 'atom(relation(usedFor,4,6,8,10)).', 'atom(relation(compare,8,10,17,19)).']
pred:  ['atom(entity(method,4,6),"0.7582648396492004").', 'atom(entity(otherScientificTerm,17,19),"0.49037060141563416").']
revi:  ['atom(entity(method,4,6)).', 'atom(entity(otherScientificTerm,17,19)).']
revi != pred:  True
------------------------
664
grtr:  ['atom(entity(method,3,5)).', 'atom(entity(method,7,10)).', 'atom(relation(usedFor,3,5,7,10)).']
pred:  ['atom(entity(method,3,5),"0.8967345356941223").', 'atom(entity(method,7,10),"0.9943734407424927").', 'atom(entity(task,14,15),"0.99449366331

------------------------
905
grtr:  ['atom(entity(task,0,2)).', 'atom(entity(otherScientificTerm,3,5)).', 'atom(entity(method,6,14)).', 'atom(entity(method,15,18)).', 'atom(entity(method,20,22)).', 'atom(relation(usedFor,3,5,0,2)).', 'atom(relation(conjunction,3,5,6,14)).', 'atom(relation(usedFor,6,14,0,2)).', 'atom(relation(conjunction,6,14,15,18)).', 'atom(relation(usedFor,15,18,0,2)).', 'atom(relation(conjunction,15,18,20,22)).', 'atom(relation(usedFor,20,22,0,2)).']
pred:  ['atom(entity(method,0,2),"0.7328047752380371").', 'atom(entity(otherScientificTerm,3,5),"0.9694888591766357").', 'atom(entity(method,6,9),"0.9965354204177856").', 'atom(entity(otherScientificTerm,10,14),"0.9796119332313538").', 'atom(entity(method,15,18),"0.9970086216926575").', 'atom(entity(method,20,22),"0.9943196773529053").', 'atom(relation(usedFor,3,5,0,2),"0.999565064907074").', 'atom(relation(conjunction,3,5,6,9),"0.9923027753829956").', 'atom(relation(usedFor,6,9,0,2),"0.9638754725456238").', 'atom(relat

revi:  ['atom(entity(method,2,5)).', 'atom(entity(otherScientificTerm,16,18)).', 'atom(relation(usedFor,16,18,2,5)).']
revi != pred:  True
------------------------
1242
grtr:  []
pred:  ['atom(entity(otherScientificTerm,9,10),"0.9602425694465637").', 'atom(entity(otherScientificTerm,12,15),"0.7963005304336548").', 'atom(relation(featureOf,12,15,9,10),"0.5875756144523621").']
revi:  ['atom(entity(otherScientificTerm,9,10)).', 'atom(entity(otherScientificTerm,12,15)).', 'atom(relation(featureOf,12,15,9,10)).']
revi != pred:  True
------------------------
1243
grtr:  ['atom(entity(otherScientificTerm,3,5)).', 'atom(entity(metric,7,8)).', 'atom(entity(otherScientificTerm,10,12)).', 'atom(relation(featureOf,7,8,10,12)).']
pred:  ['atom(entity(metric,4,5),"0.7263518571853638").', 'atom(entity(metric,7,8),"0.9802653789520264").', 'atom(entity(otherScientificTerm,9,12),"0.9680497646331787").', 'atom(relation(conjunction,4,5,7,8),"0.9149699211120605").', 'atom(relation(usedFor,7,8,9,12),"0.4016

In [192]:
count

144

In [193]:
len(unlabeled)

1303